# Previsão de Deputados Eleitos

- 1 Há desbalanceamento das classes (isto é, uma classe tem muito mais instâncias que outra)? 
    Em que proporção? 
    Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador? 
    Como você poderia tratar isso? (2 pt.)

- 2 Treine: um modelo de regressão logística, uma árvore de decisão e um modelo de adaboost. 
    Tune esses modelos usando validação cruzada e controle overfitting se necessário, 
    considerando as particularidades de cada modelo.  (2 pts.)
    
- 3 Reporte acurácia, precision, recall e f-measure no treino e validação. 
    Há uma grande diferença de desempenho no treino/validação? 
    Como você avalia os resultados? Justifique sua resposta. (2 pt.)
    
- 4 Interprete as saídas dos modelos. 
    Quais atributos parecem ser mais importantes de acordo com cada modelo? 
    Crie pelo menos um novo atributo que não está nos dados originais e estude o impacto desse atributo (2 pts.)
    
- 5 Envie seus melhores modelos à competição do Kaggle. 
    Faça pelo menos uma submissão. 
    Sugestões para melhorar o modelo: (2 pts.)
        
    - 5.1 Experimente outros modelos (e.g. SVM, RandomForests e GradientBoosting).
    - 5.2 Experimente balancear as classes, caso estejam desbalanceadas.
    - 5.3 Crie pelo menos um novo atributo.

In [2]:
%matplotlib inline

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-whitegrid')

In [3]:
def normalizando_dados(dataframe):
    
    numeric_features = dataframe.dtypes[dataframe.dtypes != "object"].index

    skewed_features = dataframe[numeric_features].apply(lambda x: x.dropna().skew())

    skewed_features = skewed_features[skewed_features > 0.75]

    skewed_features = skewed_features.index

    dataframe[skewed_features] = np.log1p(dataframe[skewed_features])

    dataframe = pd.get_dummies(dataframe)

    dataframe = dataframe.fillna(dataframe.mean())
    
    return dataframe

### Importando dados das eleições de 2014

In [5]:
eleicoes_2014 = pd.read_csv('/home/lsp/dados/am/eleicoes_2014.csv')
eleicoes_2014 = normalizando_dados(eleicoes_2014)
eleicoes_2014 = eleicoes_2014.drop(['sq_candidato', 'media_receita', 'media_despesa'], axis=1)
eleicoes_2014.head()

,ano,qtd_doacoes,qtd_doadores,total_receita,recursos de outros candidatos/comitês,recursos de pessoas físicas,recursos de pessoas jurídicas,recursos próprios,qtd_despesas,qtd_fornecedores,...,descricao_ocupacao_TÉCNICO DE QUÍMICA,descricao_ocupacao_TÉCNICO EM AGRONOMIA E AGRIMENSURA,descricao_ocupacao_TÉCNICO EM EDIFICAÇÕES,descricao_ocupacao_TÉCNICO EM INFORMÁTICA,descricao_ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"descricao_ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",descricao_ocupacao_VEREADOR,descricao_ocupacao_VETERINÁRIO,descricao_ocupacao_VIGILANTE,descricao_ocupacao_ZOOTECNISTA
0,2014,1.386294,1.386294,7.365813,8.442226,7.313887,10.039024,8.505303,1.386294,1.386294,...,0,0,0,0,0,0,0,0,0,0
1,2014,1.791759,1.791759,8.064951,8.442226,8.039480,10.039024,8.505303,1.945910,1.791759,...,0,0,0,0,0,0,0,0,1,0
2,2014,3.713572,3.663562,12.716781,7.562198,11.092869,11.945818,11.652696,4.983607,4.941642,...,0,0,0,0,0,0,0,0,0,0
3,2014,3.401197,3.401197,11.962218,10.574474,11.138739,9.741027,8.826758,4.919981,4.804021,...,0,0,0,0,0,0,0,0,0,0
4,2014,5.081404,4.990433,13.474546,9.210440,12.757293,12.531776,11.264477,6.251904,5.872118,...,0,0,0,0,0,0,0,0,0,0
